In [1]:
import os
import sys
import inspect

sys.path.insert(1, os.path.join(sys.path[0], '..'))

import fatapi
from fatapi.data import Data
from fatapi.model import BlackBox, Model
from fatapi.methods import FACEMethod
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=100, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf.predict_proba(X_test[:1])
print(clf.predict(X_test[:5, :]))
clf.score(X_test, y_test)
print(X_test.shape)
print(y_test.shape)
data_X = Data(dataset=X_test, dtype="data", encoded=True)
data_y = Data(dataset=y_test, dtype="target", encoded=True)

# Has to return a boolean 
def conditionf(X1, X2, Y1, Y2):
    return True

row_indicies = [0,1,2,3,4]
blackb = BlackBox(clf)
face_model = Model(data_X, data_y, blackbox=blackb)
face_method = FACEMethod(factuals=data_X.get_rows_as_data(row_indicies), factuals_target=data_y.get_rows_as_data(row_indicies), model=face_model, kernel_type="KDE", t_distance=0.3, t_prediction=0.5, t_density=0.01, conditions=conditionf)
print(face_method.explain())

[1 0 1 0 1]
(25, 20)
(25,)
[1.]
[0.]
[0.]


ValueError: Missing arguments in check_edge:    prediction 